<a href="https://colab.research.google.com/github/milton-miranda/Classificacao-desbalanceada/blob/main/Base2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.package(c("tidyverse", "MASS", "caTools", "caret", "ggplot2", "UBL",
                  "smotefamily", "boot", "ROCR", "pROC", "gridExtra", "DMwR",
                  "e1071", "mltools", "tictoc"))

In [ ]:
library(tidyverse)
library(MASS)
library(caTools)
library(caret)
library(ggplot2)
library(UBL)
library(smotefamily)
library(boot)
library(ROCR)
library(pROC)
library(gridExtra)
library(DMwR)
library(e1071)
library(mltools)
library(tictoc)

# OBS: Busca pelo melhor ponto de corte, nessa base, est? apenas l? em baixo (fui rodando e fazendo l? em baixo)

###### descritiva ######

base_analise <- rbind(base_treino_base2,base_teste_base2)
base_analise$Resposta <- as.factor(base_analise$Resposta)
novos_nomes <- c("Propor??o limite utilizado", "Idade", "Atraso 30-59", "Propor??o pagamento de d?vidas", "Quantidade empr?stimos ativos", "Quantidade empr?stimos imobili?rio", "Resposta")
colnames(base_analise) <- novos_nomes

graficos_descritva <- function(base_dados){

  graficos <- list()
  Indice = NULL
  Variavel = NULL
  Tipo = NULL

  for (i in 1:(length(base_dados)-1)) {
    if (class(pull(base_dados,i))== "factor" |
        class(pull(base_dados,i))== "character"){
      dados_grafico <- data.frame(Resposta = base_dados$Resposta,
                                  Variavel = pull(base_dados,i))
      graficos[[i]] <- ggplot(data = dados_grafico,
                              aes(x = Variavel,
                                  y = prop.table(stat(count)),
                                  fill = Resposta,
                                  label = scales::percent(prop.table(stat(count)),
                                                          accuracy = 1))) +
        geom_bar(position=position_dodge()) +
        geom_text(stat = 'count',
                  position = position_dodge(.9),
                  vjust = -0.5,
                  size = 3.5) +
        scale_y_continuous(labels = scales::percent)+
        scale_fill_manual(values = c("darkolivegreen4", "tomato2"),
                          labels=c("Adimplente", "Inadimplente"))+
        labs(x = "", y = 'Frequência relativa (%)', fill = "Classificação")+
        ggtitle(names(base_dados[i]))+
        theme_bw()
      Variavel[i] = names(base_dados)[i]
      Tipo[i] = "Categorica"
    }else{
      dados_grafico <- data.frame(Resposta = base_dados$Resposta,
                                  Variavel = pull(base_dados,i))
      graficos[[i]] <- ggplot(data = dados_grafico, aes(x= Resposta, y = Variavel, fill = Resposta)) +
        geom_boxplot(color = "dimgray") +
        scale_x_discrete(labels=c("Adimplente", "Inadimplente")) +
        scale_fill_manual(values=c("darkolivegreen4", "tomato2"))+
        labs(x = "",y= names(base_dados[i]))+
        ggtitle(names(base_dados[i]))+
        theme_bw()+
        theme(legend.position= "none")
      Variavel[i] = names(base_dados)[i]
      Tipo[i] = "Numérica"
    }
    Indice[i]= i
  }
  guia_graficos <-  data.frame(Indice,Variavel,Tipo)
  lista_graficos <- list(graficos=graficos,guia_graficos = guia_graficos)
  return(lista_graficos)
}

plot_graficos <- graficos_descritva(base_dados = base_analise)
grid.arrange(plot_graficos$graficos[[1]], plot_graficos$graficos[[2]], plot_graficos$graficos[[3]], plot_graficos$graficos[[4]],plot_graficos$graficos[[5]],plot_graficos$graficos[[6]], nrow = 3, ncol = 2)
grid.arrange(plot_graficos$graficos[[1]], plot_graficos$graficos[[2]], nrow = 1, ncol = 2)
grid.arrange(plot_graficos$graficos[[3]], plot_graficos$graficos[[4]], nrow = 1, ncol = 2)
grid.arrange(plot_graficos$graficos[[5]], plot_graficos$graficos[[6]], nrow = 1, ncol = 2)


###################
# treino e teste ? separado assim pois os arquivos j? vieram separados
treino <- base_treino_base2
teste <- base_teste_base2
treino$Resposta <- as.factor(treino$Resposta)
teste$Resposta <- as.factor(teste$Resposta)

treino_sub <- treino_subamostrado_base2
treino_sub$Resposta <- as.factor(treino_sub$Resposta)
names(treino_sub)[names(treino_sub) == "Unnamed..0"] <- "X"
treino_sub <- subset(treino_sub, select = -X)
############ REGRESSAO LOGISTICA DESBALANCEADA ##################
#regressao logistica no conjunto de treinamento
library(corrplot)
base_analise_numeric <- select_if(base_analise, is.numeric)
correlation_matrix <- cor(base_analise_numeric)
corrplot(correlation_matrix, method = "color")
cor(base_analise$Atraso30_59,base_analise$Prop_limite_utilizado)
tic()
model <- glm(Resposta ~.,family=binomial(link='logit'),data=treino)
toc()
summary(model)
#predicao no conjunto de teste
p_samp <- predict(model, newdata=subset(teste), type="response")
p_samp

predicoes_totais <- predict(model, type = "response")
classe1 <- (treino$Resposta == 1)
predicoes_classe1 <- predicoes_totais[classe1]
summary(predicoes_classe1)
boxplot(model$fitted.values ~ treino$Resposta)

#KS
pred_modelo_geral = prediction(p_samp, teste$Resposta)

perf_modelo_geral = performance(pred_modelo_geral,"tpr","fpr")

TPRfromROCR=unlist(perf_modelo_geral@y.values)
FPRfromROCR=unlist(perf_modelo_geral@x.values)

diff_TPRFPR=TPRfromROCR-FPRfromROCR
KS=max(diff_TPRFPR)
cutoffAtKS=unlist(perf_modelo_geral@alpha.values)[which.max(diff_TPRFPR)]
print(c(KS, cutoffAtKS)) #0.05483515

#matriz de cunfusao
previsoes_binarias <- ifelse(p_samp > cutoffAtKS, '1', '0')
C_Matrix_Geral <- confusionMatrix(data = as.factor(previsoes_binarias),
                                  reference = as.factor(teste$Resposta),
                                  positive = '1',
                                  mode = "everything");C_Matrix_Geral
g_media <- sqrt(C_Matrix_Geral$byClass["Sensitivity"]*C_Matrix_Geral$byClass["Specificity"]);g_media
#MCC
teste_2 <- data.frame(p_samp, teste$Resposta)
class_predict <- ifelse(teste_2$p_samp > 0.5,1,0) %>% as.factor()
MCC1 <- mcc(preds = class_predict, actuals =  teste_2$teste.Resposta);MCC1
############ REGRESSAO LOGISTICA SUBAMOSTRADA ##################
#regressao logistica no conjunto de treinamento
tic()
model <- glm(Resposta ~.,family=binomial(link='logit'),data=treino_sub)
toc()
summary(model)
#predicao no conjunto de teste
p_samp <- predict(model, newdata=subset(teste), type="response")
p_samp

#KS
pred_modelo_geral = prediction(p_samp, teste$Resposta)

perf_modelo_geral = performance(pred_modelo_geral,"tpr","fpr")

TPRfromROCR=unlist(perf_modelo_geral@y.values)
FPRfromROCR=unlist(perf_modelo_geral@x.values)

diff_TPRFPR=TPRfromROCR-FPRfromROCR
KS=max(diff_TPRFPR)
cutoffAtKS=unlist(perf_modelo_geral@alpha.values)[which.max(diff_TPRFPR)]
print(c(KS, cutoffAtKS)) #0.05746823

#matriz de cunfusao
previsoes_binarias <- ifelse(p_samp > 0.14, '1', '0')
C_Matrix_Geral <- confusionMatrix(data = as.factor(previsoes_binarias),
                                  reference = as.factor(teste$Resposta),
                                  positive = '1',
                                  mode = "everything");C_Matrix_Geral
g_media <- sqrt(C_Matrix_Geral$byClass["Sensitivity"]*C_Matrix_Geral$byClass["Specificity"]);g_media
#MCC
teste_2 <- data.frame(p_samp, teste$Resposta)
class_predict <- ifelse(teste_2$p_samp > 0.14,1,0) %>% as.factor()
MCC1 <- mcc(preds = class_predict, actuals =  teste_2$teste.Resposta);MCC1
############ REGRESSAO LOGISTICA SMOTE ##################
tic()
data_balanced_smote_3 <- SmoteClassif(Resposta ~., treino, C.perc = list('0' = 1, '1' = 14))
toc()
table(data_balanced_smote_3$Resposta)
tic()
modelo_smote <- glm(Resposta ~.,family=binomial(link='logit'),data=data_balanced_smote_3)
toc()
summary(modelo_smote)
#predicao no conjunto de teste
p_samp_smote <- predict(modelo_smote, newdata=subset(teste), type="response")
p_samp_smote

predicoes_totais <- predict(modelo_smote, type = "response")
classe1 <- (data_balanced_smote_3$Resposta == 1)
predicoes_classe1 <- predicoes_totais[classe1]
summary(predicoes_classe1)
boxplot(modelo_smote$fitted.values ~ data_balanced_smote_3$Resposta)

#KS
pred_modelo_geral_smote = prediction(p_samp_smote, teste$Resposta)

perf_modelo_geral_smote = performance(pred_modelo_geral_smote,"tpr","fpr")

TPRfromROCR_smote=unlist(perf_modelo_geral_smote@y.values)
FPRfromROCR_smote=unlist(perf_modelo_geral_smote@x.values)

diff_TPRFPR_smote=TPRfromROCR_smote-FPRfromROCR_smote
KS_smote=max(diff_TPRFPR_smote)
cutoffAtKS_smote=unlist(perf_modelo_geral_smote@alpha.values)[which.max(diff_TPRFPR_smote)]
print(c(KS_smote, cutoffAtKS_smote)) #0.4774525

#matriz de cunfusao
previsoes_binarias_smote <- ifelse(p_samp_smote > 0.65, '1', '0')
C_Matrix_Geral_smote <- confusionMatrix(data = as.factor(previsoes_binarias_smote),
                                        reference = teste$Resposta,
                                        positive = '1',
                                        mode = "everything");C_Matrix_Geral_smote
g_media_smote <- sqrt(C_Matrix_Geral_smote$byClass["Sensitivity"]*C_Matrix_Geral_smote$byClass["Specificity"]);g_media_smote
teste_smote <- data.frame(p_samp_smote, teste$Resposta)
class_predict_smote <- ifelse(teste_smote$p_samp_smote > 0.65,1,0) %>% as.factor()
MCC_smote <- mcc(preds = class_predict_smote, actuals =  teste_smote$teste.Resposta);MCC_smote
############ REGRESSAO LOGISTICA HIBRIDA ##################
table(treino_sub$Resposta)
tic()
data_balanced_hibrido <- SmoteClassif(Resposta ~., treino_sub, C.perc = list('0' = 1, '1' = 13.5))
toc()
table(data_balanced_hibrido$Resposta)

#modelo com base balanceada
tic()
modelo_h <- glm(Resposta ~.,family=binomial(link='logit'),data=data_balanced_hibrido)
toc()
summary(modelo_h)
#predicao no conjunto de teste
p_samp_h <- predict(modelo_h, newdata=subset(teste), type="response")
p_samp_h
#KS
pred_modelo_geral_h = prediction(p_samp_h, teste$Resposta)

perf_modelo_geral_h = performance(pred_modelo_geral_h,"tpr","fpr")

TPRfromROCR_h=unlist(perf_modelo_geral_h@y.values)
FPRfromROCR_h=unlist(perf_modelo_geral_h@x.values)

diff_TPRFPR_h=TPRfromROCR_h-FPRfromROCR_h
KS_h=max(diff_TPRFPR_h)
cutoffAtKS_h=unlist(perf_modelo_geral_h@alpha.values)[which.max(diff_TPRFPR_h)]
print(c(KS_h, cutoffAtKS_h)) #0.4831057

#matriz de cunfusao
previsoes_binarias_h <- ifelse(p_samp_h > 0.73, '1', '0')
C_Matrix_Geral_h <- confusionMatrix(data = as.factor(previsoes_binarias_h),
                                    reference = teste$Resposta,
                                    positive = '1',
                                    mode = "everything");C_Matrix_Geral_h
g_media_h <- sqrt(C_Matrix_Geral_h$byClass["Sensitivity"]*C_Matrix_Geral_h$byClass["Specificity"]);g_media_h
teste_h <- data.frame(p_samp_h, teste$Resposta)
class_predict_h <- ifelse(teste_h$p_samp_h > 0.73,1,0) %>% as.factor()
MCC_h <- mcc(preds = class_predict_h, actuals =  teste_h$teste.Resposta);MCC_h
############ SVM DESBALANCEADA ##################
tic()
svm_radial <- fit_svm(treino,teste,prob = F, kernel ="radial")
toc()
df.svm.radial <- data.frame(Predicao = svm_radial$saida_modelo,Y = teste$Resposta)
performance.svm.radial <- fit_performance(data = df.svm.radial,prob = F);performance.svm.radial

############ SVM SMOTE ##################
tic()
svm_radial_smote <- fit_svm(data_balanced_smote_3,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_smote <- data.frame(Predicao = svm_radial_smote$saida_modelo,Y = teste$Resposta)
performance.svm.radial_smote <- fit_performance(data = df.svm.radial_smote,prob = F);performance.svm.radial_smote

############ SVM subamostragem ##################
#treino_sub <- subset(treino_sub, select = -X)
tic()
svm_radial_sub <- fit_svm(treino_sub,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_sub <- data.frame(Predicao = svm_radial_sub$saida_modelo,Y = teste$Resposta)
performance.svm.radial_sub <- fit_performance(data = df.svm.radial_sub,prob = F);performance.svm.radial_sub

############ SVM HIBRIDO ##################
#data_balanced_hibrido <- subset(data_balanced_hibrido, select = -X)
tic()
svm_radial_h <- fit_svm(data_balanced_hibrido,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_h <- data.frame(Predicao = svm_radial_h$saida_modelo,Y = teste$Resposta)
performance.svm.radial_h <- fit_performance(data = df.svm.radial_h,prob = F);performance.svm.radial_h

############ SVM SENSIVEL AO CUSTO ##################
tic()
svm_peso <- SVM_weight(treino,teste,kernel = "radial", prob = TRUE, peso_min = 13,
                       peso_max = 14, range = 0.2)
toc()
svm_peso_resultados <- SVM_weight_perf(svm_peso,prob = FALSE);svm_peso_resultados
svm_peso_resultados$G_Mean
svm_peso_resultados$F1
svm_peso_resultados$MCC


#########################################################
#Para encontrar o melhor ponto de corte, nessa base, ia rodando o p_samp nos c?digos acima e colocando aqui em baixo
########## BUSCA MELHOR PTCO CORTE
calcular_MCC_para_pontos_corte <- function(data, A, ponto_corte_seq) {
  melhores_MCC <- -Inf
  melhor_ponto_corte <- NA

  for (ponto_corte in ponto_corte_seq) {
    class_predict <- ifelse(data$p_samp_h > ponto_corte, 1, 0) %>% as.factor()
    MCC <- mcc(preds = class_predict, actuals = A$teste.Resposta)

    if (MCC > melhores_MCC) {
      melhores_MCC <- MCC
      melhor_ponto_corte <- ponto_corte
    }
  }

  return(list(melhor_ponto_corte = melhor_ponto_corte, melhor_MCC = melhores_MCC))
}
# Exemplo de uso:
teste_2 <- data.frame(p_samp_h, teste$Resposta)
ponto_corte_seq <- seq(0, 0.99, 0.01)
resultado <- calcular_MCC_para_pontos_corte(teste_2, teste_2, ponto_corte_seq)

print(paste("Melhor Ponto de Corte:", resultado$melhor_ponto_corte))
print(paste("Melhor Valor de MCC:", resultado$melhor_MCC))
